## 1. 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

data_columns = train.columns.drop(['ID','y'])

print(f'train shape : {train.shape}')
print(f'test shape : {test.shape}')

train shape : (40118, 13)
test shape : (4986, 12)


## 2. 데이터 전처리

### 이상치 처리

In [2]:
# # IQR을 사용한 이상치 탐지 및 제거 함수 정의
# def remove_outliers(df, column):
#     # 1사분위수(25%)와 3사분위수(75%) 계산
#     Q1 = df[column].quantile(0.25)
#     Q3 = df[column].quantile(0.75)
#     # IQR 계산
#     IQR = Q3 - Q1
#     # 이상치 범위 설정 (1.5 * IQR보다 더 크거나 작은 값들)
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
#     # 이상치가 아닌 데이터만 반환
#     return df[(df[column] >= lower_bound)] 
#             #   & (df[column] <= upper_bound)]

# # target 컬럼에서 이상치 제거
# train = remove_outliers(train, 'y')
# print(f'train shape : {train.shape}')

In [3]:
# from scipy import stats

# def remove_outliers_zscore(df, column, threshold=3):
#     """
#     Z-score를 사용하여 이상치를 제거하는 함수

#     Parameters:
#     df (DataFrame): 데이터 프레임
#     column (str): 이상치를 제거할 열 이름
#     threshold (float): 이상치로 간주할 Z-score의 임계값 (기본값은 3)

#     Returns:
#     DataFrame: 이상치가 제거된 데이터 프레임
#     """
#     # Z-score 계산
#     z_scores = stats.zscore(df[column])
#     # Z-score가 threshold보다 작은 데이터만 반환
#     return df[(z_scores >= -threshold)& (z_scores <= threshold)]

# # target 컬럼에서 이상치 제거
# train = remove_outliers_zscore(train, 'y')
# print(f'train shape : {train.shape}')

### 로그 변환

In [4]:
# from sklearn.preprocessing import PowerTransformer

# # Yeo-Johnson 변환기 생성
# pt = PowerTransformer(method='yeo-johnson')

# # ID와 y를 제외한 나머지 컬럼에 대해 Yeo-Johnson 변환 수행
# train_yeo = train.copy()
# train_yeo[data_columns] = pt.fit_transform(train[data_columns])
# test_yeo = test.copy()
# test_yeo[data_columns] = pt.transform(test[data_columns])

In [5]:
# train_yeo = train_yeo.drop(columns = ['ID','x_8','y'])
# test_yeo = test_yeo.drop(columns = ['ID','x_8'])
# train_y = train['y']

### 다중공선성

In [6]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.linear_model import LinearRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.tools.tools import add_constant
# import lightgbm as lgb

# # VIF 계산 함수
# def calculate_vif(X):
#     X = add_constant(X)
#     vif = pd.DataFrame()
#     vif["Feature"] = X.columns
#     vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
#     return vif.iloc[1:, :]  # const 제외


# # train/test split
# X_train, X_test, y_train, y_test = train_test_split(train, train_y, test_size=0.25, random_state=42)

# # 초기 모델 학습 및 평가
# model = CatBoostRegressor(random_state=42, verbose=300)

# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# initial_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
# print(f"Initial RMSE: {initial_rmse}")

# # VIF 계산 및 시각화
# vif_data = calculate_vif(train)
# print(vif_data)

# # 각 피쳐를 제거하면서 VIF 및 모델 성능 비교
# vif_scores = []
# rmse_scores = []
# features = train.columns.tolist()

# for feature in features:
#     X_train_reduced = X_train.drop(columns=[feature])
#     X_test_reduced = X_test.drop(columns=[feature])
    
#     # 모델 학습 및 평가
#     model.fit(X_train_reduced, y_train)
#     y_pred_reduced = model.predict(X_test_reduced)
#     rmse_reduced = np.sqrt(mean_squared_error(y_test, y_pred_reduced))
    
#     # VIF 계산
#     vif_reduced = calculate_vif(X_train_reduced)
    
#     # 결과 저장
#     vif_scores.append(vif_reduced['VIF'].mean())
#     rmse_scores.append(rmse_reduced)

# # 시각화
# plt.figure(figsize=(12, 6))

# plt.subplot(1, 2, 1)
# plt.plot(features, vif_scores, marker='o')
# plt.title('VIF Score by Removing Each Feature')
# plt.xlabel('Feature Removed')
# plt.ylabel('Mean VIF')

# plt.subplot(1, 2, 2)
# plt.plot(features, rmse_scores, marker='o', color='orange')
# plt.title('RMSE by Removing Each Feature')
# plt.xlabel('Feature Removed')
# plt.ylabel('RMSE')

# plt.tight_layout()
# plt.show()


In [7]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from sklearn.decomposition import PCA

# # 3. 다중공선성 처리 (VIF 계산)
# X = train.drop(columns=['ID','y'])
# vif_data = pd.DataFrame()
# vif_data["feature"] = X.columns
# vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# print(vif_data)
# print(vif_data.VIF.sum())

### 시각화

In [8]:
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Create histograms for each variable
# fig, axes = plt.subplots(4, 3, figsize=(20, 20))
# axes = axes.ravel()
 
# for i, col in enumerate(train_yeo_johnson.columns[1:]):  # Skip the ID column
#     sns.histplot(train_yeo_johnson[col], ax=axes[i], kde=True)
#     axes[i].set_title(f'Distribution of {col}')

# plt.tight_layout()
# plt.show()

# # Create box plots for each variable
# fig, axes = plt.subplots(4, 3, figsize=(20, 20))
# axes = axes.ravel()

# for i, col in enumerate(train_yeo_johnson.columns[1:]):  # Skip the ID column
#     sns.boxplot(y=train_yeo_johnson[col], ax=axes[i])
#     axes[i].set_title(f'Box plot of {col}')

# plt.tight_layout()
# plt.show()

### 피쳐 생성

In [9]:
# import sweetviz as sv
# train_sv = train.drop(columns=['ID',])
# # Sweetviz 리포트 생성
# report = sv.analyze(train_sv)

# # 리포트 출력
# report.show_html('sweetviz_report.html')  

In [16]:
train_f = train.drop(columns=['ID','y'])
test_f = test.drop(columns=['ID'])
train_y = train.y

In [17]:
# 3. 데이터 스케일링
scaler = StandardScaler()
train_scaler = pd.DataFrame(scaler.fit_transform(train_f), columns=train_f.columns)
test_scaler = pd.DataFrame(scaler.transform(test_f), columns=test_f.columns)

In [18]:
train_scaler['x_7_x_9'] = train_scaler['x_7'] * train_scaler['x_9']
train_scaler['x_8 * x_9'] = train_scaler['x_8'] * train_scaler['x_9']
# train_scaler['x_8_plus_x_9'] = train_scaler['x_8'] + train_scaler['x_9'] #2.9086
train_scaler['x_8_minus_x_9'] = train_scaler['x_8'] - train_scaler['x_9'] #2.9078
# train_scaler['x_4_div_x_5'] = train_scaler['x_4'] / train_scaler['x_5']


test_scaler['x_7_x_9'] = test_scaler['x_7'] * test_scaler['x_9']
test_scaler['x_8 * x_9'] = test_scaler['x_8'] * test_scaler['x_9']
# test_scaler['x_8_plus_x_9'] = test_scaler['x_8'] + test_scaler['x_9'] #2.9086
test_scaler['x_8_minus_x_9'] = test_scaler['x_8'] - test_scaler['x_9'] #2.9078
# test_scaler['x_4_div_x_5'] = test_scaler['x_4'] / test_scaler['x_5']


train_scaler = train_scaler.drop(columns=['x_0','x_5','x_6'])
test_scaler = test_scaler.drop(columns=['x_0','x_5','x_6'])
# 0 2.8390
# 1 2.8509
# 2 2.8479
# 3 2.8375
# 4 2.8373 
# 5 2.8332 #
# 6 2.8356 ###
# 7 2.8377
# 8 2.8448
# 9 2.8346 ##
# 10 2.8366

# 5 6 9 2.8323
# 5 6 2.8298


In [19]:
# import shap
# from sklearn.model_selection import train_test_split
# from catboost import CatBoostRegressor
# from sklearn.model_selection import cross_val_score

# X_train, X_test, y_train, y_test = train_test_split(train_scaler, train_y, test_size=0.2, random_state=42)
# X_train = X_train.drop(columns=['x_2'])
# X_test = X_test.drop(columns=['x_2'])
# # 모델 학습
# model = CatBoostRegressor(random_state=42, verbose=0)
# model.fit(X_train, y_train)

# # SHAP 해석기 생성
# explainer = shap.TreeExplainer(model)  # 트리 기반 모델에 적합한 해석기
# shap_values = explainer.shap_values(X_test)

# # SHAP 값을 데이터프레임으로 변환
# shap_values_df = pd.DataFrame(shap_values, columns=X_test.columns)

# # 각 피쳐별로 SHAP 값의 절대값을 구한 후, 평균 계산
# mean_absolute_shap_values = shap_values_df.abs().mean()

# # 결과 출력
# mean_absolute_shap_values_sorted = mean_absolute_shap_values.sort_values(ascending=False)
# print(mean_absolute_shap_values_sorted)

In [20]:
# # 모델 학습 및 교차 검증
# model = CatBoostRegressor(random_state=42, verbose=0)
# scores = cross_val_score(model, train_scaler, train_y, cv=5, scoring='neg_mean_squared_error')

# print(f"Cross-validated MSE: {-scores.mean():.4f}") # 2.9077

## 3. 분석 모델 설계

In [21]:
import xgboost as xgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error # root_mean_squared_error로 교체
from optuna.integration import XGBoostPruningCallback

# 학습/테스트 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(train_scaler, train_y, test_size=0.3, random_state=42)

def objective(trial):
    # 하이퍼파라미터 샘플링
    param = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'lambda': trial.suggest_float('lambda', 1e-6, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-6, 1.0, log=True),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 0.9),
        'subsample': trial.suggest_float('subsample', 0.5, 0.9),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.2, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 1),
        'random_state': 42,
        'early_stopping_rounds': 100,  # 조기 종료 설정
    }

    # 모델 학습
    model = xgb.XGBRegressor(**param)
    model.fit(X_train, y_train,
              eval_set=[(X_test, y_test)],
              verbose=False)

    # 예측 및 평가
    preds = model.predict(X_test)
    rmse = root_mean_squared_error(y_test, preds)  # root_mean_squared_error로 대체

    return rmse

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=500)

trial = study.best_trial
xgb_best_params = trial.params


[I 2024-08-28 18:02:36,450] A new study created in memory with name: no-name-f6a16f2e-ae9c-429d-bccc-fea92b96cb80
[I 2024-08-28 18:02:36,828] Trial 0 finished with value: 1.6400173738075123 and parameters: {'lambda': 0.19892230508385156, 'alpha': 2.1151449355195534e-05, 'colsample_bytree': 0.756645451218194, 'subsample': 0.649737915764707, 'learning_rate': 0.15813114982956689, 'n_estimators': 228, 'max_depth': 8, 'min_child_weight': 10, 'gamma': 0.20344734510786933}. Best is trial 0 with value: 1.6400173738075123.
[I 2024-08-28 18:02:43,924] Trial 1 finished with value: 1.6532924443324168 and parameters: {'lambda': 0.0021468939841983756, 'alpha': 0.0009806561424947878, 'colsample_bytree': 0.8940704961569663, 'subsample': 0.8896976155591538, 'learning_rate': 0.005472100218869935, 'n_estimators': 878, 'max_depth': 11, 'min_child_weight': 2, 'gamma': 0.056635040545244375}. Best is trial 0 with value: 1.6400173738075123.
[I 2024-08-28 18:02:44,724] Trial 2 finished with value: 2.4731062627

In [22]:
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import Ridge

estimators = [
    ('lr', LinearRegression()),
    ('xgb', xgb.XGBRegressor(**xgb_best_params)),
]

# 스태킹 회귀 모델
stacking_model = StackingRegressor(estimators=estimators, final_estimator=RandomForestRegressor(n_estimators=300,random_state=42))
stacking_model.fit(train_scaler, train_y)

StackingRegressor(estimators=[('lr', LinearRegression()),
                              ('xgb',
                               XGBRegressor(alpha=0.005124748801861577,
                                            base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=0.8762812403544114,
                                            device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None,
                                            gamma=0.32371107744912...
                                            lambda=0.3409912401290758,
                                            learning_rate=0.04220871807939122,
                                            max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=3,
                                            max_leaves=None,
                                            min_child_weight=10, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=731, n_jobs=None, ...))],
                  final_estimator=RandomForestRegressor(n_estimators=300,
                                                        random_state=42))

## 4. 모델 학습

In [23]:
# import matplotlib.pyplot as plt

# # 모델 학습 후 특성 중요도 추출
# importances = rf.feature_importances_
# indices = np.argsort(importances)[::-1]

# # 특성 중요도 시각화
# plt.figure(figsize=(10, 6))
# plt.title("Feature Importance")
# plt.bar(range(train_x_scaled.shape[1]), importances[indices], align="center")
# plt.xticks(range(train_x_scaled.shape[1]), train_x_scaled.columns[indices], rotation=90)
# plt.tight_layout()
# plt.show()

## 5. 예측값 생성
학습한 모델을 사용하여 예측값을 생성합니다.

In [24]:
# 예측 및 평가
y_pred = stacking_model.predict(test_scaler)

## 5. 제출 파일 생성
submission 파일을 만들어서 제출합니다.

In [25]:
from datetime import datetime

submission = pd.read_csv('sample_submission.csv')

# 현재 날짜와 시간을 'YYYYMMDD_HHMMSS' 형식으로 포맷팅
today_datetime = datetime.today().strftime('%Y%m%d_%H%M')
submission['y'] = y_pred

# 포맷팅한 날짜와 시간으로 파일 이름 생성
submission.to_csv(f'./submission_{today_datetime}.csv', index=False)

In [26]:
submission.head()

,ID,y
0,TEST_0000,82.901877
1,TEST_0001,81.502325
2,TEST_0002,90.582405
3,TEST_0003,90.369535
4,TEST_0004,83.236600


In [27]:
import winsound

# 주파수와 지속시간 설정 (주파수 단위: Hertz, 지속시간 단위: 밀리초)
frequency = 1000  # 주파수 (Hertz)
duration = 300    # 지속시간 (Milliseconds)

# 소리 재생
winsound.Beep(frequency, duration)